# Reconstructing the path
In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: /data/twitter/twitter_sample_small.txt

Format: user_id \t follower_id

You can start with the code described in "Starter..." (see the next self-reading).

Your task is to find the shortest path between vertices 12 and 34. In case of multiple shortest paths (that is, disjoint paths with the same length), any will suffice. Output format is the sequence of vertices, delimited by __a comma, without spaces.__ For example, the path “12 -> 42 -> 34” should be printed as:

    12,42,34

_Hint: before submitting, check your stopping criteria. In BFS, the search was exhaustive, and in this task your program may terminate earlier, thus saving some precious time._

The result on the sample dataset:

    12,422,53,52,107,20,23,274,34

If you want to deploy the environment on your own machine, please use [bigdatateam/spark-course1](https://hub.docker.com/r/bigdatateam/spark-course1/) Docker container.

In [11]:
!cat "/data/twitter/twitter_sample_small.txt" | awk '/422\t12/'
!cat "/data/twitter/twitter_sample_small.txt" | awk '/53\t422/'
!cat "/data/twitter/twitter_sample_small.txt" | awk '/31\t53/'

422	12
53	422
31	53


In [2]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

import re

In [3]:
n = 100
start = 12

def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    # Add one move along the graph
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    # cast next_v to [next_v] and append to the paths
    return (next_v, prev_d + [next_v])

edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()
# construct paths for (start, [start, node1, node2...])


In [12]:
print edges.take(5)
print forward_edges.take(5)
print paths.take(1)
print paths.join(forward_edges, n).take(1)
print paths.join(forward_edges, n).map(step).take(1)

[(12, 2241), (12, 13349), (12, 41873), (12, 82473), (12, 414853)]
[(16447200, 12), (16929600, 12), (17204700, 12), (17981800, 12), (19177100, 12)]
[(53, [12, 422, 53])]
[(53, ([12, 422, 53], 31))]
[(31, [12, 422, 53, 31])]


In [13]:
end = 34
paths = sc.parallelize([(start, [start])]).partitionBy(n)
def found():
    print paths.collect(), paths.filter(lambda x: x[0] == end).count()
    return paths.filter(lambda x: x[0] == end).count()

while not found():
    paths = paths.join(forward_edges, n).map(step)
    
paths.cache()

[(12, [12])] 0
[(126, [12, 126]), (380, [12, 380]), (422, [12, 422]), (648, [12, 648])] 0
[(53, [12, 422, 53])] 0
[(31, [12, 422, 53, 31]), (52, [12, 422, 53, 52]), (57, [12, 422, 53, 57]), (150, [12, 422, 53, 150]), (187, [12, 422, 53, 187]), (292, [12, 422, 53, 292]), (652, [12, 422, 53, 652])] 0
[(13, [12, 422, 53, 31, 13]), (107, [12, 422, 53, 52, 107])] 0
[(15, [12, 422, 53, 52, 107, 15]), (20, [12, 422, 53, 52, 107, 20]), (586, [12, 422, 53, 52, 107, 586])] 0
[(23, [12, 422, 53, 52, 107, 20, 23]), (87, [12, 422, 53, 52, 107, 20, 87]), (292, [12, 422, 53, 52, 107, 20, 292]), (295, [12, 422, 53, 52, 107, 20, 295]), (107, [12, 422, 53, 52, 107, 586, 107])] 0
[(15, [12, 422, 53, 52, 107, 586, 107, 15]), (20, [12, 422, 53, 52, 107, 586, 107, 20]), (586, [12, 422, 53, 52, 107, 586, 107, 586]), (16, [12, 422, 53, 52, 107, 20, 23, 16]), (38, [12, 422, 53, 52, 107, 20, 23, 38]), (274, [12, 422, 53, 52, 107, 20, 23, 274]), (64, [12, 422, 53, 52, 107, 20, 87, 64])] 0
[(23, [12, 422, 53, 52,

PythonRDD[320] at collect at <ipython-input-13-7b9209de030c>:4

In [18]:
print paths.collect()

[(23, [12, 422, 53, 52, 107, 586, 107, 20, 23]), (87, [12, 422, 53, 52, 107, 586, 107, 20, 87]), (292, [12, 422, 53, 52, 107, 586, 107, 20, 292]), (295, [12, 422, 53, 52, 107, 586, 107, 20, 295]), (17, [12, 422, 53, 52, 107, 20, 23, 274, 17]), (34, [12, 422, 53, 52, 107, 20, 23, 274, 34]), (107, [12, 422, 53, 52, 107, 586, 107, 586, 107])]


In [19]:
result = (paths
         .filter(lambda value: value[0] == end)
         .map(lambda value: value[1])
        ).cache()
print result.collect()

[[12, 422, 53, 52, 107, 20, 23, 274, 34]]


In [14]:
# while True:
#     candidates = distances.join(forward_edges, n).map(step).persist()
#     new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
#     target = new_distances.filter(lambda x: x[0] == end)
#     if target.count() > 0:
#         print target.take(5)
#         spath = target.collect()[0][1]
#         break  
    
#     count = new_distances.filter(lambda i: i[1] == d + 1).count()
#     if count > 0:
#         d += 1
#         distances = new_distances
#     else:
#         break